In [67]:
import pandas as pd
import numpy as np
import time

#upload dataset
train_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/train.csv", header = None, names = ['variance','skewness','curtosis','entropy','y'])
test_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/test.csv", names = ['variance','skewness','curtosis','entropy','y'])

features = ['variance','skewness','curtosis','entropy']
outcome = 'y'

X_train = train_data[features].values #change to matrix multiple
y_train = train_data[outcome].values
X_test = test_data[features].values
y_test = test_data[outcome].values
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [ ]:
class PrimalSVM:
    def __init__(self, gamma, a, C, N):
        """
        Initialize the SVM model with hyperparameters.
        """
        self.gamma = gamma
        self.a = a
        self.C = C
        self.N = N
        self.w = None  # Weight vector will be initialized during training

    def fit(self, X, y, epochs, schedule):
        n_features = X.shape[1]
        self.w = np.zeros(n_features)  # Initialize weights to zeros

        for epoch in range(epochs):
            data = np.column_stack((X, y))
            np.random.shuffle(data)  # Shuffle training data
            X_shuffled = data[:, :-1]
            y_shuffled = data[:, -1]

            for i, (xi, yi) in enumerate(zip(X_shuffled, y_shuffled)):
                t = epoch * len(y_shuffled) + i + 1  # Global step count
                
                # Learning rate schedules
                if schedule == "schedule1":
                    gamma_t = self.gamma / (1 + (self.gamma / self.a) * t)
                elif schedule == "schedule2":
                    gamma_t = self.gamma / (1 + t)
                else:
                    raise ValueError("Invalid schedule. Choose 'schedule1' or 'schedule2'.")


                if yi * np.dot(self.w, xi) <= 1:
                    self.w = self.w - gamma_t * self.w + gamma_t * self.C * self.N * yi * xi
                else:
                    self.w = (1 - gamma_t) * self.w

    def predict(self, X):
        return np.sign(X.dot(self.w))

    def score(self, X, y):
        predictions = self.predict(X)
        errors = np.mean(predictions != y)
        return errors


In [76]:
# Define hyperparameter search space
Cs = [100 / 873, 500 / 873, 700 / 873, 873]
gamma_values = [0.1, 0.01, 0.001]
a_values = [1, 10, 100]
N = 1
epochs = 100


lowest_error = 10000
best_params = None

for C in Cs:
    for gamma in gamma_values:
        for a in a_values:
            svm = PrimalSVM(gamma=gamma, a=a, C=C, N=N)
            
            svm.fit(X_train, y_train, epochs=epochs, schedule="schedule2")
            train_errors = svm.score(X_train, y_train)
            test_errors = svm.score(X_test, y_test)
            print(f"Testing: C={C}, gamma={gamma}, a={a}, train_error: {train_errors}, test_error:{test_errors}")

            if test_errors < lowest_error:
                lowest_error = test_errors
                best_params = (C, gamma, a)

print(f"Best parameters: C={best_params[0]}, gamma={best_params[1]}, a={best_params[2]}")
print(f"Best accuracy: {lowest_error}")

Testing: C=0.1145475372279496, gamma=0.1, a=1, train_error: 0.19839449541284404, test_error:0.196
Testing: C=0.1145475372279496, gamma=0.1, a=10, train_error: 0.2018348623853211, test_error:0.196
Testing: C=0.1145475372279496, gamma=0.1, a=100, train_error: 0.1823394495412844, test_error:0.176
Testing: C=0.1145475372279496, gamma=0.01, a=1, train_error: 0.25688073394495414, test_error:0.252
Testing: C=0.1145475372279496, gamma=0.01, a=10, train_error: 0.30504587155963303, test_error:0.308
Testing: C=0.1145475372279496, gamma=0.01, a=100, train_error: 0.25688073394495414, test_error:0.264
Testing: C=0.1145475372279496, gamma=0.001, a=1, train_error: 0.31880733944954126, test_error:0.326
Testing: C=0.1145475372279496, gamma=0.001, a=10, train_error: 0.31995412844036697, test_error:0.328
Testing: C=0.1145475372279496, gamma=0.001, a=100, train_error: 0.33142201834862384, test_error:0.334
Testing: C=0.572737686139748, gamma=0.1, a=1, train_error: 0.06077981651376147, test_error:0.076
Testi

In [ ]:
class DuallSVM:
    def __init__(self, gamma, a, C, N):
        """
        Initialize the SVM model with hyperparameters.
        """
        self.gamma = gamma
        self.a = a
        self.C = C
        self.N = N
        self.w = None  # Weight vector will be initialized during training

    def fit(self, X, y, epochs, schedule):
        n_features = X.shape[1]
        self.w = np.zeros(n_features)  # Initialize weights to zeros

        for epoch in range(epochs):
            data = np.column_stack((X, y))
            np.random.shuffle(data)  # Shuffle training data
            X_shuffled = data[:, :-1]
            y_shuffled = data[:, -1]

            for i, (xi, yi) in enumerate(zip(X_shuffled, y_shuffled)):
                t = epoch * len(y_shuffled) + i + 1  # Global step count
                
                # Learning rate schedules
                if schedule == "schedule1":
                    gamma_t = self.gamma / (1 + (self.gamma / self.a) * t)
                elif schedule == "schedule2":
                    gamma_t = self.gamma / (1 + t)
                else:
                    raise ValueError("Invalid schedule. Choose 'schedule1' or 'schedule2'.")


                if yi * np.dot(self.w, xi) <= 1:
                    self.w = self.w - gamma_t * self.w + gamma_t * self.C * self.N * yi * xi
                else:
                    self.w = (1 - gamma_t) * self.w

    def predict(self, X):
        return np.sign(X.dot(self.w))

    def score(self, X, y):
        predictions = self.predict(X)
        errors = np.mean(predictions != y)
        return errors


500

TypeError: fit() missing 1 required positional argument: 'epochs'